## The Language

The standard foundations of Mathematics is a **formal theory** consisting of the following. 

1. Two **alphabets**, one for **special symbols** like "&forall;" and "&exist;" and another for **variable** names like "x", "y", "z".
2. Syntax rules for excluding meaningless sentences like "∀ x (y:z) ∀ ∀ ∃", and for defining **free variables** and closed formulas without free variables. 
3. **Deduction rules** for constructing closed formulas from closed formulas.
4. A collection of closed formulas called **axioms**.

The axioms are meant to be "true" intuitively so that &forall; is supposed to behave like what we think of as "for all", etc.  For example, the Zermelo-Fraenkel set theory with the Axiom of Choice (ZFC) has ten axioms.  This is the most often cited foundation of Mathematics meaning that mathematical results should be reducible, in principle, to valid deductions in ZFC. 

Coda is meant to be an alternative to this as a foundation.  Rather than assuming or using predicate logic syntax, the foundation of Coda is **finite sequence**.  From this, one defines "pure" data and coda.  Logic in is then internal, derived and "2+ valued", where `()` and `(:)` are the iconic "true" and "false" valuea and where there are many "undecided" data values like `(foo:bar)`.  Coda has only one axiom - it is the axiom that defines what constitutes a valid definition.  The Coda language itself is merely one definition like any other.  This means that we don't need any additional axioms or deduction rules. 

In Coda, a definition is just a partial function from codas to data.  The Coda language, for instance, is a partial function from coda to data, acting on codas of the form ({...some source code...} A : B) where A and B are any data.  There are essentially only two operations in coda:

1. Concatenation of data A and data B, written `A B`.
2. Creating a coda from data A and data B, written `A:B`.

and the essential idea of the language is to make these two operations specifyable in text string source code ({...some source code...} A : B) and to get programmatic access to the "A" data (sometimes called "argument data") and the "B" data (sometimes called "input").  Thus, a sketch of the internal definition is:

* `{x : y} A : B` &rightarrow; `({x} A : B) : ({y} A : B)` ..to specify forming a coda from two data 
* `{x   y} A : B` &rightarrow; `({x} A : B)   ({y} A : B)` ..to specify concatenation of two data 
* `{A} A : B` &rightarrow; `A` ..to get the "A part" of the coda.
* `{B} A : B` &rightarrow; `B` ..to get the "B part" of the coda.

Because the compiler is merely a standard definition, the meaning of the language does not need a special explanation or additional axioms.  We don't even have to define proper syntax.  Coda has the unusual property that all finite sequences of bytes are valid language expressions.  There is no such thing as a syntax error. 

The full compiler is tiny, easily readable, and can be found in Language.py.  The full compiler and parser is only about 100 lines of Python.

In [1]:
#
#   Coda uses spaces to denote concatenation and parenthesis to group operations.
#   Comment lines must start with # are considered to be comments.
#
a b (c d) e (f g)

a b c d e f g

In [2]:
#
#   There are basically two operations: concatenation and colon. here, 
#   for instance, we reverse the order of 1 2 3 using rev.  This works by first 
#   creating the coda (rev : 1 2 3).  A definition applies to this coda, which 
#   results the "input" to rev in reverse order 3 2 1.  
#
rev : 1 2 3 

3 2 1

In [3]:
#
#   Some "commands" like rev have arguments.  first, for example, gets 
#   the first items from a sequence. 
#
first 2 : a b c d e 

a b

In [4]:
#
#   Colons can be combined. The default grouping is from the right.  This is first 2 : (rev : a b c d e).
#   (first 2 : rev ) : a b c d e is completely different.  Intuitively, you can think of data flowing 
#   from the right to the left with a b c d e consumed by rev, and then, the result gets consumed 
#   by "first 2:".
#
first 2 : rev : a b c d e 

e d

In [5]:
#
#   Lines are continued by indentation.
#
first 2 : rev : a b 
   c d e

e d

In [6]:
#
#   Lines without indentation are the same as (line1) (line2), etc.  
#
first 2 : rev : a b
c d e

b a
c d e

In [7]:
#
#   Text in curly braces is interpreted as Coda language expressions.  "B" in the language, is a special symbol 
#   which refers to the "input", in this case, the data a b c d e... 
#
{first 2 : B} : a b c d e 

a b

In [8]:
#
#   The general language coda is ({...} A : B)  where "A" in the language referes to A and "B" refers to B.
#
{A B A B} 1 2 : x y 

1 2 x y 1 2 x y

In [9]:
#
#   So you can do...
#
{first A : B} 2 : a b c d e 

a b

In [10]:
#
#   You can use "step" to show how this get's evaluated in more detail.  
#   Each step below is made by application definitions.  You can see 
#   that Coda freely mixes data and source code as it computes.  
#
step : {first A : B} 2 : a b c d e

[0] (({{first A : B} 2 }:):({ a b c d e}:))
[1] (({{first A : B} 2}:):({a b c d e}:))
[2] (({{first A : B}}:) ({2}:):({a}:) ({b c d e}:))
[3] (({first A } 2:a ({b}:) ({c d e}:)):({ B} 2:a ({b}:) ({c d e}:)))
[4] (({first A} 2:a b ({c}:) ({d e}:)):({B} 2:a b ({c}:) ({d e}:)))
[5] (({first} 2:a b c ({d}:) ({e}:)) ({A} 2:a b c ({d}:) ({e}:)):a b c ({d}:) ({e}:))
[6] a (first 1:b c d e)
[7] a b (first 0:c d e)
[8] a b

In [11]:
#
#   You can use "def" to make your own definitions.  This one defines a new 
#   command "mydef" which gets the first n elements from the reversed input. 
#
def mydef : {first A : rev : B} 
mydef 2 : a b c d e

e d

In [12]:
#
#   Notice how Coda handles "undecided" data like (foo:bar) where 
#   no current definition acts on (foo:...)...
#
rev : 1 2 3 (foo:bar)

(rev:(foo:bar)) 3 2 1

In [13]:
#
#   (foo:bar) is like a "variable", i.e. like something 
#   that may have a value in the future.  If (foo:bar) gets defined later, 
#   then (rev:(foo:bar)) 3 2 1 will evaluate to the correct answer while, at the same 
#   time, current computations will be correct, for instance... 
#
last 2 : rev : 1 2 3 (foo:bar)

2 1

In [14]:
#
#    Sequences in Coda are always finite sequences, but this isn't 
#    a real limitation.  The natural numbers, for instance are embodied by 
#    a definition (nat:n) -> n (nat:n+1).  This means that (nat:0) will get 
#    evaluated as necessary.  Although this is finite,  
#
(nat:0)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 (nat:98)

In [15]:
#
#    What's the "reverse of the natural numbers?".  Let's see... 
#
rev : nat : 0

(rev:(nat:96)) (rev:95) 94 93 92 91 90 89 88 87 86 85 84 83 82 81 80 79 78 77 76 75 74 73 72 71 70 69 68 67 66 65 64 63 62 61 60 59 58 57 56 55 54 53 52 51 50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 0

In [16]:
#
#...And, for example, you can reverse twice to get the original natural numbers back.
#
rev : rev : nat : 0 

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 (rev:(rev:92)) (rev:(rev:(nat:93)))

In [17]:
#
#   Notice, that, as expected, the "first element of the reverse of the 
#   natural numbers" is not defined.  Yet, the last element is perfectly OK.
#
last : rev : nat : 0 

0

In [18]:
#
#   defs: produces a list of current definitions in context and 
#   help : first, etc. gives help information and demos which 
#   you can execute by hand or via the "demo" command.
#
help : first

code:
    first
module:
    Sequence
summary:
    Gets the A-specified leading items from input data.
description:
    first   : B -> (first 1:B)
    first 0 : B -> ()
    first n : L R -> L (first n-1 : R) L is atom
path:
    /Users/youssef/coda/py/Sequence.py
demos:
    1. first : a b c
    2. first 1 : a b c
    3. first 2 : a b c
    4. {first 3:B} : {first 2:B} : 1 2 3 4 5
    5. first 3 : first 2 : 1 2 3 4 5


In [19]:
demo 4 : first 

1 2

### Combinatorics 

Any system of reasoning must contain undefined terms.  In the case of Coda, the **finite sequence** is the foundational concept that is assumed to be understood without definition.  It's pleasing and expected, then, that the low level operations in Coda are things are the natural low level combinatorial operations on finite sequences.  We've looked at two of these as simple examples:

* `rev` reverses the order of a sequence. 
* `first` gets the first n items of a sequence. 

Some of the most important operations are combinatorics of the form (name A:B) where A gets used in various combinatoric ways on B.  The most important one of these is called `ap`.  The idea is that something like this 

* `ap A : b1 b2 b3...bn`

is defined to be "A applied to each b1,...bn".  It's defined to be (A:b1) (A:b2)...(A:bn).  The actual definition of ap is 

* `ap A : B C -> (ap A:B) (ap A:C)`
* `ap A : B -> (A:B)` if B is an atom 
* `ap A : () -> ()` 

which is the same idea, applicable to all "input" data (B) and all "arguments" (A).  Coda has a collection of these named definitions built into the system.  They are all simple combinations of the three ingredients``(name A : B)`.  These are basic definitions underlying many of the more familiar mathematical objects built on top.  As far as I know, these don't have conventional names, so we are forced into naming them.

| coda | schematic result |
|------|------------|
|  `ap` A:b1 b2 b3  | (A:b1) (A:b2) (A:b3) |
|  `aq` a1 a2 a3:B | (a1:B) (a2:B) (a3:B) |
| `ax` a a1 a2 a3:B | (a a1:B) (a a2:B) (a a3:B)  |
| `by` 2 A:b1 b2 b3 b4... | (A:b1 b2) (A:b2 b3)... |

We will do a few examples below.  Each has a help entry and examples which can be run in the notebook or command line.  Although the names are not familiar, they are so simple that an example or two is an effective way to learn them. 

In [20]:
#
#   Each has a help entry with examples.
#   You can run the demos by hand or with the demo command. 
#
help : ap 

code:
    ap
module:
    Apply
summary:
    Apply A to each b in B.
description:
    This is one of the most important basic
    combinatorial operations.  ap A is guaranteed
    to be distributive data for any data A.
    ap A : B C -> (ap A:B) (ap A:C)
    ap A : B -> A:B if B is atom
    ap A : () -> ()
path:
    /Users/youssef/coda/py/Apply.py
demos:
    1. ap foo : 1 2 3
    2. ap {foo : B} : 1 2 3
    3. ap {first A : get bin : B} 2 : (bin:a b c d e) (bin:x y z)
    4. aq a b c d : 1 2 3 4
    5. ap aq a b c : 1 2 3
    6. ax a b c : 1 2 3
    7. ax first 2 3 : a b c d e  g
    8. ap ax a b c : 1 2 3
    9. by 2 foo : a b c d e f g
    10. ap {if (count:get bin:B)=2:B} : (bin:a b) (bin:a b c) (bin:x y) (bin:a b c d)


In [21]:
#
#  The meaning of each is easy to understand by testing with sample inputs. 
#
ap a b c : 1 2 3 
aq a b c d : 1 2 3 
ax f a b c : 1 2 3 
by 2 f a1 a2 : 1 2 3 

(a b c:1) (a b c:2) (a b c:3)
(a:1 2 3) (b:1 2 3) (c:1 2 3) (d:1 2 3)
(f a:1 2 3) (f b:1 2 3) (f c:1 2 3)
(f a1 a2:1 2) (f a1 a2:3)

## Use of **if** 

(if A : B) has the meaning B if A is true.  For example...

In [22]:
if () : 1 2 3 4 
if (:) : 1 2 3 4 

1 2 3 4

In [23]:
#
#   if can be nicely combined with other things. 
#
ap {if (B=2):B} : a b c 1 2 a b c 2 2 
ap {if ((domain:B)=bin) : right:B} : 1 2 (bin:x y z) (bin: a b c)

2 2 2
x y z a b c

## Use of the \* operator 

The \* operation is defined by A\*B:X = A:B:X for any data A, B, and X.  This means that \* is associative since (A\*B)\*C:X = A\*(B\*C):X = A:B:C:X always.  Recall that the colon binds to the right first so A:B:C:X is A:(B:(C:X)).  

In [24]:
#
#   Let's define "revfirstn" to compute the first n items of the reversed input. 
#   For example with n=3...  
#
first 3 : rev : 1 2 3 4 5 6 7

7 6 5

In [25]:
#
#   sometimes * can be used to avoid curly braces as in these two equivalent definitions 
#
def revfirstn : {first A : rev : B} 
def revfirstN : first A * rev  

In [26]:
#
#   the above two work the same 
#
revfirstn 3 : 1 2 3 4 5 6 7 8 
revfirstN 3 : 1 2 3 4 5 6 7 8

8 7 6
8 7 6

In [27]:
#
#   A few dozen definitions are built into the system and enter via python modules. 
#   the rest come from .co coda source code files.
#
defs:

ap...............Apply.....3..Apply A to each b in B.
aq...............Apply.....3..Apply A to each b in B.
ax...............Apply.....2..Apply A to each b in B.
by...............Apply.....2..Apply A to each b in B.
ker..............Apply.....1..ker A is the kernel of A defined by A:X=().
kernel...........Apply.....1..ker A is the kernel of A defined by A:X=().
mor..............Apply.....1..Morphism space operation
while............Apply.....1..while A : B applies A to B until it saturates, thus becoming idempotent
*................Basic.....1..Star is syntactic sugar with A*B:X defined to be A:B:X
bin..............Basic.....0..pass : B -> B and null : B -> ()
get..............Basic.....1..has/hasnt selects, get gets, is/isnt tests
has..............Basic.....1..has/hasnt selects, get gets, is/isnt tests
hasnt............Basic.....1..has/hasnt selects, get gets, is/isnt tests
if...............Basic.....2..if and nif return output depending on argument logic.
is...............Basic.....1

## Constants

Suppose you want some data, say the sequence 1 2 3 to be a convenient constant that you can use in multiple situations.  You can always do 

let x : 1 2 3 

This creates a definition (x:) &rightarrow; 1 2 3 which can be evalued by the coda source code **(x:)**, or by the equivalent **x?**. 

In [28]:
#
#   You can also assign a data values to any undefined coda.  For convenience, 
#   the compiler takes x? and makes this actually (?:x) to make convenient things that 
#   look like "variables" but are actually codas.  
#
let x : 1 2 3 
(x:)
x?

1 2 3
1 2 3

## Domains and undefine data 

Definitions are partial functions from codas to data.  The partial function (if any) which applies to a coda is determined by the **domain** of the coda, which is accessible via "domain."  

In [34]:
domain : a b c ((:):(:)) (bin:x y z)
domain : x?
domain : y?
undefined : a b c (foo:bar) x? y?
undefined : (bin: bin : bin : z? z? foo?) 

𝝞 𝝞 𝝞 ◎ bin
𝝞 𝝞 𝝞
(domain:(y:))
(y:) (foo:bar)
(z:) (foo:)

## While and looping 

Most of the "looping" within coda is implicit, but one can also use **while** where 

while A : B 

means A:B, A:A:B, A:A:A:B,... evaluating to a result when the iteration stops changing.  This guarantees that while A is idempotent for any A.  Here's an example.  

In [30]:
#... to sum a list of integers, you can use int_sum 

int_sum : 1 2 3 4 5 6 7 8 9 10 

55

In [31]:
#... using by 2, we can sum the sequence two at a time 

by 2 int_sum : 1 2 3 4 5 6 7 8 9 10 

3 7 11 15 19

In [32]:
#.... with a while in front, the sequence is summed by two repeatedly until saturation.

while by 2 int_sum : 1 2 3 4 5 6 7 8 9 10

55